# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [13]:
with open("car_sale_qa.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [14]:
from langchain.text_splitter import CharacterTextSplitter

In [31]:
text_splitter = CharacterTextSplitter(        
    separator = r'---',
    chunk_size = 115,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [32]:
docs = text_splitter.create_documents([real_estate_sales])

In [33]:
docs[0]
docs[1]

Document(page_content='[客户问题] 我担心新车会不会很快就贬值。\n[销售回答] 我完全理解您的担忧。不过，这款车型因其耐用性和品牌信誉，在二手市场上保值率非常高。我们还提供多种金融方案来帮助您保护投资。')

In [34]:
len(docs)

107

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [35]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [36]:
query = "车费不费油"

In [37]:
answer_list = db.similarity_search(query)

In [38]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我不知道是否应该选择柴油车还是汽油车。 
[销售回答] 每种类型的车都有其独特的优势。柴油车通常油耗更低，适合长途驾驶；汽油车则动力更加平顺，适合城市驾驶。根据您的驾驶习惯和需求，我们可以一起找到最适合您的选择。

[客户问题] 这款车的油耗怎么样？
[销售回答] 这款车采用了最新的节能技术，平均油耗仅为X升/百公里，不仅能满足您对经济性的需求，长期来看还能为您节省大量的油费。

[客户问题] 我想要一辆节能减排的车。 
[销售回答] 这款车的发动机采用了最新的节能技术，不仅提高了燃油效率，还大幅度减少了排放，帮助您做到环保节能。

[客户问题] 我想要一辆有智能启停系统的车。
[销售回答] 这款车配备了智能启停技术，能在车辆暂停时自动熄火，减少燃油消耗和排放，再次出发时迅速启动，既环保又经济。



In [39]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [40]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [41]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B14BC80250>, search_kwargs={'k': 3})

In [42]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 我不知道是否应该选择柴油车还是汽油车。 
[销售回答] 每种类型的车都有其独特的优势。柴油车通常油耗更低，适合长途驾驶；汽油车则动力更加平顺，适合城市驾驶。根据您的驾驶习惯和需求，我们可以一起找到最适合您的选择。

[客户问题] 这款车的油耗怎么样？
[销售回答] 这款车采用了最新的节能技术，平均油耗仅为X升/百公里，不仅能满足您对经济性的需求，长期来看还能为您节省大量的油费。

[客户问题] 我想要一辆节能减排的车。 
[销售回答] 这款车的发动机采用了最新的节能技术，不仅提高了燃油效率，还大幅度减少了排放，帮助您做到环保节能。



In [43]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪车啊？")

In [44]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我想要一辆内饰豪华的车。 
[销售回答] 这款车的内饰采用了高档材料，如真皮座椅、木质饰板和金属装饰，提供了多种内饰颜色和风格选择，确保您能享受到豪华舒适的驾驶环境。

[客户问题] 我需要一辆适合高速公路驾驶的车。 
[销售回答] 这款车的动力充沛，高速稳定性极佳，配备了巡航控制系统，长时间高速驾驶也不会感到疲劳。

[客户问题] 我想要一辆有高级音响系统的车。 
[销售回答] 这款车配备了品牌高级音响系统，音质清晰丰富，无论是古典音乐还是现代电子，都能为您提供沉浸式的听觉享受。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [45]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5}
)

In [46]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我不知道是否应该选择柴油车还是汽油车。 
[销售回答] 每种类型的车都有其独特的优势。柴油车通常油耗更低，适合长途驾驶；汽油车则动力更加平顺，适合城市驾驶。根据您的驾驶习惯和需求，我们可以一起找到最适合您的选择。

[客户问题] 这款车的油耗怎么样？
[销售回答] 这款车采用了最新的节能技术，平均油耗仅为X升/百公里，不仅能满足您对经济性的需求，长期来看还能为您节省大量的油费。

[客户问题] 我想要一辆节能减排的车。 
[销售回答] 这款车的发动机采用了最新的节能技术，不仅提高了燃油效率，还大幅度减少了排放，帮助您做到环保节能。

[客户问题] 我想要一辆有智能启停系统的车。
[销售回答] 这款车配备了智能启停技术，能在车辆暂停时自动熄火，减少燃油消耗和排放，再次出发时迅速启动，既环保又经济。



### 提取向量数据库中的`销售回答`

In [47]:
docs = retriever.get_relevant_documents(query)

In [48]:
docs[0].page_content

'[客户问题] 我不知道是否应该选择柴油车还是汽油车。 \n[销售回答] 每种类型的车都有其独特的优势。柴油车通常油耗更低，适合长途驾驶；汽油车则动力更加平顺，适合城市驾驶。根据您的驾驶习惯和需求，我们可以一起找到最适合您的选择。'

In [49]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我不知道是否应该选择柴油车还是汽油车。 \n',
 '每种类型的车都有其独特的优势。柴油车通常油耗更低，适合长途驾驶；汽油车则动力更加平顺，适合城市驾驶。根据您的驾驶习惯和需求，我们可以一起找到最适合您的选择。']

In [52]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [53]:
ans

'每种类型的车都有其独特的优势。柴油车通常油耗更低，适合长途驾驶；汽油车则动力更加平顺，适合城市驾驶。根据您的驾驶习惯和需求，我们可以一起找到最适合您的选择。'

#### 尝试各种问题

In [54]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [55]:
query = "我想要便宜一点，好看好开的车"

print(sales(query))

[]


C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [56]:
print(sales(query, 0.75))

['这款车的设计融合了经典与现代元素，线条流畅，外观时尚大方，非常符合现代审美标准。', '这款车因其可靠性和品牌声誉，在二手市场上的保值率非常高。我们还提供车辆维护记录，帮助您在未来保持车辆的价值。', '这款车的尺寸适中，操控灵活，非常适合城市驾驶。它的燃油效率高，能在繁忙的城市交通中为您节省时间和费用。', '这款车的动力充沛，高速稳定性极佳，配备了巡航控制系统，长时间高速驾驶也不会感到疲劳。']


In [57]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['这款车的定价考虑了市场竞争和性能配置的平衡。我们提供的是最优质的产品和服务，确保您的投资物有所值，并且我们可以讨论不同的付款方案来满足您的预算需求。', '这款车因其可靠性和品牌声誉，在二手市场上的保值率非常高。我们还提供车辆维护记录，帮助您在未来保持车辆的价值。', '这款车型非常受欢迎，市场需求稳定，所以转手相对容易。我们的品牌信誉和车辆的可靠性都是保证二手价值的重要因素。', '我完全理解您的担忧。不过，这款车型因其耐用性和品牌信誉，在二手市场上保值率非常高。我们还提供多种金融方案来帮助您保护投资。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [58]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.4}))

In [59]:
qa_chain({"query": "你们有200万的豪车吗？"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '你们有200万的豪车吗？', 'result': '对不起，我不知道。'}

In [60]:
qa_chain({"query": "加速快不快"})

{'query': '加速快不快',
 'result': '这款车的加速性能非常出色，从0加速到100公里/小时只需要X秒。您可以在试驾中亲自体验这款车强劲的动力和迅捷的响应。'}

In [61]:
print(sales("吵不吵"))

[]


C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [62]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [63]:
from langchain.chains import RetrievalQA
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [64]:
qa_chain({"query": "我想买豪车，你们有么"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买豪车，你们有么', 'result': '对不起，我无法帮助你购买豪车。我是一个AI助手，主要用于提供信息和回答问题。'}

In [65]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [66]:
qa_chain({"query": "我想买一辆好看价格性价比高的车，你们有么"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买一辆好看价格性价比高的车，你们有么',
 'result': '对不起，我不能提供具体的购车建议，因为我是一个人工智能，没有实际的汽车销售信息。我建议你根据你的预算和需求去汽车销售店或者在线平台查看和比较不同的车型。'}

In [67]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [68]:
result = qa_chain({"query": "我想买一辆好看价格性价比高的车，你们有么"})

C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [69]:
result

{'query': '我想买一辆好看价格性价比高的车，你们有么',
 'result': '对不起，我不能为您推荐具体的车型，因为我是一个人工智能，无法了解和评估实体商品。建议您可以参考一些汽车评测网站或者咨询汽车销售专家来获取更专业的建议。',
 'source_documents': []}